In [ ]:
from coco_pipe.io import load, select_features
from coco_pipe.ml.pipeline import MLPipeline
import pandas as pd
import numpy as np


# Load your data
features_file = '/home/mat/eeg_epilepsy/filtered_demo_epilepsy.csv'
participant_file='/home/mat/projects/def-kjerbi/cocolab_data/EEG_ADHD_epilepsy_psychostimulants/BIDS/participants.tsv'
# Read CSV into a DataFrame
features = pd.read_csv(features_file)
participants_df = pd.read_csv(participant_file, sep='\t')
participants = participants_df[(participants_df['ADHD'] == '0') & (participants_df['Autism'] == '0')]


X = features
X = X.replace([np.inf, -np.inf], np.nan)

# Drop rows with NaN
X = X.dropna()
# Clip extreme values to safe range for float32
X = X.clip(lower=-1e30, upper=1e30) 

y = participants['Epilepsy']
y = y.dropna()
y = y.replace('0 (potentiel)', '0')
y = y.iloc[:len(X)]


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# convert to binary 
# y = y.replace('0 (potentiel)', '0')
#select that don't have autism or adhd 
participants = participants_df[(participants_df['ADHD'] == '0') & (participants_df['Autism'] == '0')]
# X = load(
#     type="tabular",  # Supports: 'tabular', 'embeddings', 'meeg'
#     data_path=features
# )

# # Optionally select specific features
# X, y = select_features(
#     df=X,  # Your feature DataFrame
#     target_columns=y,  # Target variable(s)
#     covariates=["age", "sex"],  # Optional demographic/clinical variables
#     spatial_units=["left_frontal", "right_frontal"],  # Brain regions/sensors
#     feature_names=["alpha", "beta"]  # Features to include
# )

# Configure and run ML pipeline
config = {
    "task": "classification",  # or 'regression'
    "analysis_type": "baseline",  # Options: 'baseline', 'feature_selection', 'hp_search', 'hp_search_fs'
    "models": "all",  # or list of specific models
    "metrics": ['accuracy', 'f1'],
    "cv_strategy": "stratified",
    "n_splits": 5,
    "n_features": 10,  # For feature selection
    "direction": "forward",  # For feature selection
    "search_type": "grid",  # For hyperparameter search
    "n_iter": 100,  # For random search
    "scoring": "accuracy",
    "n_jobs": -1
}

pipeline = MLPipeline(X=X, y=y, config=config)

results = pipeline.run()

2025-07-22 22:02:37,521 - coco_pipe.ml.base - INFO - Starting baseline evaluation for model 'Logistic Regression'...
2025-07-22 22:02:37,765 - coco_pipe.ml.base - INFO - Metric 'accuracy' - Mean: 0.5115, Std: 0.0420
2025-07-22 22:02:37,766 - coco_pipe.ml.base - INFO - Metric 'f1' - Mean: 0.5056, Std: 0.0406
2025-07-22 22:02:37,766 - coco_pipe.ml.base - INFO - Feature 'feature-BandRatiosFromAverageFooof.bands_pairs-('alpha', 'beta').spaces-C3' - Mean importance: 0.0057
2025-07-22 22:02:37,766 - coco_pipe.ml.base - INFO - Feature 'feature-BandRatiosFromAverageFooof.bands_pairs-('alpha', 'beta').spaces-C4' - Mean importance: 0.0132
2025-07-22 22:02:37,767 - coco_pipe.ml.base - INFO - Feature 'feature-BandRatiosFromAverageFooof.bands_pairs-('alpha', 'beta').spaces-Cz' - Mean importance: -0.0902
2025-07-22 22:02:37,767 - coco_pipe.ml.base - INFO - Feature 'feature-BandRatiosFromAverageFooof.bands_pairs-('alpha', 'beta').spaces-F3' - Mean importance: 0.0256
2025-07-22 22:02:37,768 - coco_pip